In [9]:
import tensorflow as tf
import mtcnn
from mtcnn import MTCNN
import PIL
from PIL import Image
import numpy as np
import pandas as pd
from IPython.display import display
import os
import matplotlib.pyplot as pyplot

In [10]:
def extract_face(filename, detector):
    #Open the File
    image = Image.open(filename)
    #print(filename)
    #Convert the file to RGB
    image = image.convert('RGB')
    
    #Convert the File to Numpy array to be machine readable
    pixels = np.asarray(image)
    
    #Extracting Face embeddings from the Photo
    result = detector.detect_faces(pixels)
    if len(result) == 0 :
        print ('Face in the photo {} could not be detected'.format(filename))
        return []
    else:
        x1, y1, width, height = result[0]['box']
        x1, y1 = abs(x1) , abs(y1)
        x2, y2 = abs(x1) + width , abs(y1) + height 
        face = pixels[y1:y2, x1:x2]
        image = Image.fromarray(face)
        image = image.resize((160,160))
        face_array = np.asarray(image)
        return face_array

In [11]:
def load_faces(directory, detector):
    faces = list()
    for filename in os.listdir(directory):
        path = directory + '/' + filename 
        face = extract_face(path, detector)
        if len(face) == 0:
            continue
        else:
            faces.append(face)
    return faces

In [12]:
def load_dataset(directory, detector):
    X, y = list(), list()
    for sub_dir in os.listdir(directory):
        path = directory + sub_dir
        faces = load_faces(path, detector)
        print ('Loaded {f} examples from class {c}'.format(f = len(faces), c = sub_dir))
        labels = [sub_dir for _ in range(len(faces))]
        X.extend(faces) 
        y.extend(labels)
    return np.asarray(X), np.asarray(y)

In [13]:
detector = MTCNN()

In [14]:
X_train, y_train = load_dataset('../data/train/', detector)

Loaded 26 examples from class Aditya Solanki
Loaded 14 examples from class Ben Afflek
Loaded 17 examples from class Elton John
Loaded 21 examples from class Jerry Seinfeld
Loaded 19 examples from class Madonna
Loaded 22 examples from class Mindy Kaling
Loaded 36 examples from class Unknown


In [17]:
X_test, y_test = load_dataset('../data/val/', detector)

Loaded 5 examples from class Aditya Solanki
Loaded 5 examples from class Ben Afflek
Loaded 5 examples from class Elton John
Loaded 5 examples from class Jerry Seinfeld
Loaded 5 examples from class Madonna
Loaded 5 examples from class Mindy Kaling
Loaded 5 examples from class Unknown


In [19]:
np.savez_compressed('train_test_data.npz', X_train, y_train, X_test, y_test)